United Nations ComTrade
=====

## Bilateral trade data by product

----

*September 3, 2017*<br>
*@bd_econ*

This example retrieves annual data for the trade of a specific product by all countries for which data are available.

[Documentation](https://comtrade.un.org/data/doc/api/) for the UN Comtrade API.

This example uses a list of country codes stored as a csv file. 

In [1]:
import requests
import pandas as pd
import time

# Used to loop over countries 5 at a time.
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in xrange(0, len(seq), size))

c_codes = pd.read_csv('codes/country_codes.csv').set_index('id')

## Paramaters/ Settings for request

In [5]:
prod_type = 'C'  # Commodity
freq = 'A'       # Annual 
classification = 'HS' # harmonized system
prod = '440710'   # HS 6-digit production ID
years = ['2005', '2010', '2015']
base = 'http://comtrade.un.org/api/get?'
url = '{}max=50000&type={}&freq={}&px={}'.format(
    base,
    prod_type,
    freq,
    classification)
d = {}

In [6]:
for n in chunker(c_codes.index.values[1:], 5):
    req = '&ps={}&r=all&p={}&rg=1%2C2&cc={}'.format(
        '%2C'.join(years),
        '%2C'.join(n), 
        prod)
    data = requests.get('{}{}'.format(url, req)).json()
    d[n[0]] = pd.DataFrame(data['dataset'])[['TradeValue', 'pt3ISO', 
                                             'rt3ISO', 'rtTitle', 
                                             'period', 'rgDesc'
                                            ]].set_index('pt3ISO')
    time.sleep(5)

In [7]:
df = pd.concat(d.values())
df.fillna(value='TWN', inplace=True)
df = df.drop('rtTitle', 1)
df.head()

,TradeValue,rt3ISO,period,rgDesc
pt3ISO,,,,
MYS,277953,ARG,2015,Export
MYS,120140,AUS,2015,Import
MYS,7349902,AUS,2015,Export
MYS,671,AUT,2015,Export
MLT,2868925,AUT,2015,Export


In [8]:
df.to_csv('440710.csv')